## Submit Job

Once your job-type has been registered and built (see: [pge_create.ipynb](pge_create.ipynb)), jobs can be submitted from python using the steps laid out in this notebook.

While this notebook only shows submission of a single job/parameter-set, you can map or iterate over a collection of input parameter sets to efficiently submit large batches of jobs.

### It also assumes you have a valid account on the cluster.

### Initialize

When prompted for the HySDS host, include the protocol, e.g. https://soamc-mozart.jpl.nasa.gov

When prompted for "HySDS cluster authenticated", enter 'y' if the cluster requires a password to access.

In [ ]:
import json
import os
import otello
import re
import shutil

from pathlib import Path
from pprint import pprint

if not os.path.exists(f"{Path.home()}/.config/otello/config.yml"):
    otello.client.initialize()

m = otello.mozart.Mozart()

### Instantiate a `JobType` object

You will need to ensure that the the tag (i.e. the portion following the colon) matches the branch name used by your repository.

You will also need to set *job_type_name* to the name of your job's notebook.

In [ ]:
job_type_name = 'job-xing-topsstack_hamsar_pge:devel'  # This will need to be customised using the relevant action name (ie job/hysds-spec suffix) and branch name.

job_type = m.get_job_types()[job_type_name]
job_type.initialize()

### Obtain useful information about the job-type

The line below lists:
- the input schema of the job-type
- type and default value of parameters

In [ ]:
job_type.get_queues()
print(job_type.describe())
#pprint(job_type.get_input_params())

### Specify values to parameters for your job

Here is where user-defined parameter values are provided

In [ ]:
# for Goma
custom_parameters = {
    "min_lat": -1.77066085473858,
    "max_lat": -1.462637710194377,
    "min_lon": 29.13133428070789,
    "max_lon": 29.38055714034091,
    "bbox": [],
    "start_date": "2021-01-01",
    "end_date": "2021-01-15",
    "track_number": 174
}

job_type.set_input_params(custom_parameters)
print("Your job will be run with these parameters:")
pprint(job_type.get_input_params())

### Submit the job

A job tag (useful for finding the job later) and job queue are specified.

Both are optional. Job submission is asynchronous, so this call will return almost immediately.

In [ ]:
from datetime import datetime

sample_job_tag = f'{datetime.strftime(datetime.now(), "%Y%m%d")}_topsstack_hamsar_test_66'
print(sample_job_tag)

#job_run = job_type.submit_job(tag=sample_job_tag, queue="factotum-job_worker-small")
job_run = job_type.submit_job(tag=sample_job_tag, queue="factotum-2-job_worker-large")


### Monitor job state

Information about the job state will print periodically, until the job is completed.

In [ ]:
job_run.wait_for_completion()